In [1]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [2]:
!pip install ipykernel==4.10
!pip install tokenizers==0.9.4


  Using cached https://files.pythonhosted.org/packages/22/5e/69b0ae84e0cd7c8d4467e0a7b390c560d8c7642493329b15461b41e20fcc/ipykernel-4.10.0-py3-none-any.whl
ERROR: pydeck 0.5.0 has requirement ipykernel>=5.1.2; python_version >= "3.4", but you'll have ipykernel 4.10.0 which is incompatible.
  Found existing installation: ipykernel 5.4.3
    Uninstalling ipykernel-5.4.3:
      Successfully uninstalled ipykernel-5.4.3


In [3]:
pip install simpletransformers

  Using cached https://files.pythonhosted.org/packages/7d/cc/e8908bbb2921732f6851ebbbe4b77b925aab62e644ab9402f21c84fa6107/ipykernel-5.4.3-py3-none-any.whl
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.3 which is incompatible.
  Found existing installation: ipykernel 4.10.0
    Uninstalling ipykernel-4.10.0:
      Successfully uninstalled ipykernel-4.10.0


In [4]:
import os 
from google.colab import drive
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/DATASET AML/PROJECT/dataset_giusti')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:

train = pd.read_csv("BINARY_augmented_EDA_undersampling_senza_stopword_FINAL.csv")
train["comment_text"] = train["comment_text"].astype(str)
train.head()

,comment_text,bad
0,comrade soviet hell,1
1,nope not read guide appeal not time user quest...,0
2,would like help think block fatuorum,0
3,mosslash wpsubpage article,0
4,yeah administrate like shit anyway close user ...,1


In [6]:

os.chdir('/content/drive/My Drive/DATASET AML/PROJECT/dataset_giusti/TEST')
test = pd.read_csv("test_binary_senza_stopword.csv")


In [7]:
train=train.rename(columns={"comment_text":"text", "bad":"labels"})
test=test.rename(columns={"comment_text":"text", "bad":"labels"})

XLNET

In [8]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=4, overwrite_output_dir= True,
                                                       max_seq_length= 100, train_batch_size= 128, eval_batch_size= 128)

# Create a ClassificationModel
model = ClassificationModel(
    "xlnet", "xlnet-base-cased", args=model_args,  num_labels=2,
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

args = { 'model_type': 'bert', 'model_name': 'bert-base-cased', 'output_dir': 'outputs/', 'cache_dir': 'cache/', 'fp16': True, 'fp16_opt_level': 'O1', 'max_seq_length': 100, 'train_batch_size': 256, 'eval_batch_size': 256, 'gradient_accumulation_steps': 1, 'num_train_epochs': 1, 'weight_decay': 0, 'learning_rate': 4e-5, 'adam_epsilon': 1e-8, 'warmup_ratio': 0.06, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'logging_steps': 50, 'evaluate_during_training': False, 'save_steps': 2000, 'eval_all_checkpoints': True, 'use_tensorboard': True, 'overwrite_output_dir': True, 'reprocess_input_data': False, }

In [9]:
# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)



  0%|          | 0/46362 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/363 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 4:   0%|          | 0/363 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/363 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/363 [00:00<?, ?it/s]

  0%|          | 0/63978 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/500 [00:00<?, ?it/s]

In [10]:
model_outputs

array([[ 3.29882812, -4.2421875 ],
       [-1.50585938,  1.93554688],
       [-0.02200317, -0.14660645],
       ...,
       [-1.94921875,  2.5234375 ],
       [-5.08984375,  4.73828125],
       [ 3.14257812, -4.12890625]])

In [11]:
result

{'auprc': 0.7476055523047691,
 'auroc': 0.9616568467829723,
 'eval_loss': 0.5850905880630016,
 'fn': 267,
 'fp': 9313,
 'mcc': 0.553839656332715,
 'tn': 48422,
 'tp': 5976}

In [12]:
wrong_predictions

 ...]